# Track Analysis: Corners, Braking & Acceleration Zones

This notebook automatically detects and visualizes track features from your telemetry data.

## What You'll Find Here

- **Corner Detection**: Automatically identifies corners from GPS curvature data, including direction (L/R), apex location, and approximate radius
- **Corner Map**: Interactive GPS map showing detected corners with apex markers
- **Braking Zone Detection**: Identifies where heavy braking occurs, averaged across your fastest laps
- **Acceleration Zone Detection**: Identifies throttle application zones, with gear-change gaps merged
- **Track Segment Visualization**: Color-coded GPS map showing braking (red), corners (orange), and acceleration (green) zones

## How It Works

1. **Corner Detection**: Computes track curvature from GPS coordinates and identifies sustained high-curvature sections
2. **Zone Averaging**: Analyzes laps within 103% of your best lap time to find consistent braking/acceleration patterns
3. **Segment Creation**: Combines corners with braking/accel zones into a complete track segmentation

## Using Your Own Data

To analyze your own data:

1. **Run the first cell** below to install packages and display the upload widget
2. **Click "Choose File"** to select your `.xrk` or `.xrz` file
3. **Tune detection parameters** (optional): The corner detection has parameters you can adjust:
   - `threshold=0.006`: Curvature threshold (~167m radius). Lower = detect gentler corners
   - `min_corner_length=15`: Minimum corner length in samples
   - `min_gap=80`: Merge same-direction corners within this distance (meters)
4. **Run all remaining cells** to analyze your data

The status indicator will show which file is being used. If you don't upload a file, the sample data will be used.

## Requirements

- GPS data channels (`GPS Latitude`, `GPS Longitude`, `GPS Speed`)
- Brake pressure (`BrakePress`) and throttle (`PPS`) for zone detection

**Note:** This notebook works in both JupyterLite (browser) and standard JupyterLab environments.

In [1]:
# Install required packages (needed for JupyterLite, skipped in regular JupyterLab if already installed)
%pip install -q pandas plotly libxrk motorsports-data-notebook jinja2 ipywidgets

# Import core libraries
import numpy as np
import plotly.graph_objects as go

# Import helper functions
from motorsports_data_notebook.channels import (
    get_best_lap_channels,
    get_top_laps,
)
from motorsports_data_notebook.corners import identify_corners
from motorsports_data_notebook.visualization import (
    format_lap_time,
    plot_track_segments,
    show_fig,
)
from motorsports_data_notebook.widgets import FileUpload, load_session
from motorsports_data_notebook.zones import create_track_segments, detect_zones_averaged

# File picker - upload your own .xrk/.xrz file or use the sample data
file_upload = FileUpload(default_file="CMD_Inferno 86_Fuji GP Sh_Generic testing_a_2248.xrz")
file_upload.display()

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load the data file with derived columns (speed_kmh, distance_m, lap_time)
log = load_session(file_upload.get_file_data())

# Get laps as pandas DataFrame
laps = log.laps.to_pandas()

/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:343: RuntimeWarning: invalid value encountered in divide
  t = np.maximum(-np.sum(SN * O, axis=1) / np.sum(SN * D, axis=1), 0)
/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:356: RuntimeWarning: divide by zero encountered in divide
  t = np.maximum(-np.sum(SN * O, axis=1) / np.sum(SN * D, axis=1), 0)
/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:357: RuntimeWarning: invalid value encountered in multiply
  dist = np.sum(np.square(O + t.reshape((len(t), 1)) * D), axis=1)


In [3]:
# ===== Channel Name Configuration =====
# Different data loggers use different channel names. Configure yours here.
# Print available channels to help identify the correct names:
print("Available channels in this log file:")
print(sorted(log.channels.keys()))

# Required channel mappings - modify values to match YOUR data logger's channel names
# These are defaults for AIM loggers
CHANNEL_NAMES = {
    # GPS channels (required for corner detection)
    "gps_latitude": "GPS Latitude",
    "gps_longitude": "GPS Longitude",
    "gps_speed": "GPS Speed",  # Speed in m/s from GPS
    # Pedal inputs (required for zone detection)
    "throttle": "PPS",  # Throttle position sensor (0-100%)
    "brake": "BrakePress",  # Brake pressure (0-100%)
    # Dynamics (optional, for throttle acceptance analysis)
    "lateral_g": "LateralAcc",  # Lateral acceleration in G
    "steering": "SteerAngle",  # Steering angle in degrees
}

Available channels in this log file:
['AmbientTemp', 'Baro', 'Best Run Diff', 'Best Today Diff', 'BrakePress', 'BrakeSw', 'CAT1', 'CH', 'ClutchSw', 'ECT', 'External Voltage', 'FL_Ch1', 'FL_Ch2', 'FL_Ch3', 'FL_Ch4', 'FL_Ch5', 'FL_Ch6', 'FL_Ch7', 'FL_Ch8', 'FR_Ch1', 'FR_Ch2', 'FR_Ch3', 'FR_Ch4', 'FR_Ch5', 'FR_Ch6', 'FR_Ch7', 'FR_Ch8', 'GPS Altitude', 'GPS Latitude', 'GPS Longitude', 'GPS Speed', 'Gear', 'InlineAcc', 'IntakeAirT', 'LF_Shock_Pot', 'LR_Shock_Pot', 'Lambda', 'LateralAcc', 'LoggerTemp', 'Luminosity', 'MAP', 'OilTemp', 'PPS', 'PitchRate', 'Predictive Time', 'Prev Lap Diff', 'RF_Shock_Pot', 'RL_Ch1', 'RL_Ch2', 'RL_Ch3', 'RL_Ch4', 'RL_Ch5', 'RL_Ch6', 'RL_Ch7', 'RL_Ch8', 'RPM', 'RR_Ch1', 'RR_Ch2', 'RR_Ch3', 'RR_Ch4', 'RR_Ch5', 'RR_Ch6', 'RR_Ch7', 'RR_Ch8', 'RR_Shock_Pot', 'Ref Lap Diff', 'RollRate', 'SpeedAverage', 'SteerAngle', 'TPMS_ALM_LF', 'TPMS_ALM_LR', 'TPMS_ALM_RF', 'TPMS_ALM_RR', 'TPMS_Press_LF', 'TPMS_Press_LR', 'TPMS_Press_RF', 'TPMS_Press_RR', 'TPMS_Temp_LF', 'TPMS_Tem

In [4]:
# Display lap times table
laps.style.format({"lap_time": format_lap_time})

,num,start_time,end_time,lap_time
0,0,0,150454,2:30.454
1,1,150454,279602,2:09.148
2,2,279602,406240,2:06.638
3,3,406240,532797,2:06.557
4,4,532797,659282,2:06.485
5,5,659282,787773,2:08.491
6,6,787773,913776,2:06.003
7,7,913776,1041397,2:07.621
8,8,1041397,1168322,2:06.925
9,9,1168322,1294676,2:06.354


In [5]:
# Extract best lap channel data using libxrk 0.5.0 methods
gps_lat_ch = CHANNEL_NAMES["gps_latitude"]
gps_lon_ch = CHANNEL_NAMES["gps_longitude"]
best_lap, channels = get_best_lap_channels(
    log, laps, [gps_lat_ch, gps_lon_ch, "speed_kmh", "distance_m"]
)

# Filter to best lap and resample to GPS timebase for visualization
best_lap_num = int(best_lap["num"])
aligned = (
    log.filter_by_lap(best_lap_num)
    .select_channels([gps_lat_ch, gps_lon_ch, "speed_kmh", "distance_m"])
    .resample_to_channel(gps_lat_ch)
    .channels
)

# Convert to arrays for corner detection and visualization
lap_channels = {
    "GPS Latitude": aligned[gps_lat_ch].column(gps_lat_ch).to_numpy(),
    "GPS Longitude": aligned[gps_lon_ch].column(gps_lon_ch).to_numpy(),
    "speed_kmh": aligned["speed_kmh"].column("speed_kmh").to_numpy(),
    "distance_m": aligned["distance_m"].column("distance_m").to_numpy(),
}

In [6]:
# Identify corners directly from GPS coordinates
# identify_corners handles GPS->XY conversion, curvature computation, and corner detection
corners = identify_corners(
    lat=lap_channels["GPS Latitude"],
    lon=lap_channels["GPS Longitude"],
    threshold=0.003,  # Tuned on Fuji and Sodegaura
    min_corner_length=15,  # Reduced to catch shorter corners
    min_gap=80,  # Merge same-direction corners within 80m
)

print(f"Found {len(corners)} corners:")
for c in corners:
    print(
        f"  {c.name} ({c.direction}): {c.start_dist:.0f}m - {c.end_dist:.0f}m (apex at {c.apex_dist:.0f}m, radius ~{c.radius:.0f}m)"
    )

Found 10 corners:
  Turn 1 (R): 702m - 960m (apex at 755m, radius ~31m)
  Turn 2 (L): 1236m - 1398m (apex at 1292m, radius ~115m)
  Turn 3 (R): 1433m - 1862m (apex at 1761m, radius ~114m)
  Turn 4 (L): 1926m - 2136m (apex at 2001m, radius ~48m)
  Turn 5 (R): 2251m - 2358m (apex at 2290m, radius ~205m)
  Turn 6 (R): 2780m - 2846m (apex at 2824m, radius ~18m)
  Turn 7 (L): 2849m - 2930m (apex at 2875m, radius ~44m)
  Turn 8 (R): 2938m - 3190m (apex at 3107m, radius ~59m)
  Turn 9 (L): 3203m - 3450m (apex at 3374m, radius ~43m)
  Turn 10 (R): 3496m - 3738m (apex at 3652m, radius ~42m)


In [7]:
# Visualize corners on GPS map with markers
fig = go.Figure()

# Plot track colored by speed
fig.add_trace(
    go.Scattermapbox(
        lat=lap_channels["GPS Latitude"],
        lon=lap_channels["GPS Longitude"],
        mode="markers",
        marker=dict(
            size=5,
            color=lap_channels["speed_kmh"],
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Speed (km/h)"),
        ),
        name="Track",
    )
)

# Add corner apex markers
for corner in corners:
    apex_idx = corner.apex_idx
    fig.add_trace(
        go.Scattermapbox(
            lat=[lap_channels["GPS Latitude"][apex_idx]],
            lon=[lap_channels["GPS Longitude"][apex_idx]],
            mode="markers+text",
            marker=dict(size=15, color="red"),
            text=[corner.name],
            textposition="top right",
            textfont=dict(size=12, color="red"),
            name=corner.name,
        )
    )

fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center=dict(
            lat=np.mean(lap_channels["GPS Latitude"]), lon=np.mean(lap_channels["GPS Longitude"])
        ),
        zoom=14,
    ),
    title="Detected Corners",
    showlegend=False,
    width=800,
    height=600,
)

show_fig(fig)

/tmp/ipykernel_2787/2716297616.py:6: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/tmp/ipykernel_2787/2716297616.py:25: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [8]:
# Identify braking and acceleration zones averaged over top laps
top_laps = get_top_laps(laps, threshold_pct=1.03)

print(f"Best lap time: {laps['lap_time'].min()}")
print(f"Using {len(top_laps)} laps within 103% of best for zone averaging")

# Detect and average zones across top laps (efficiently extracts only required channels per lap)
braking_zones, accel_zones = detect_zones_averaged(log, top_laps, CHANNEL_NAMES)

Best lap time: 0 days 00:02:05.056000
Using 13 laps within 103% of best for zone averaging


In [9]:
# Create fixed segment definitions combining corners with braking/accel zones
track_length = lap_channels["distance_m"][-1]
segments = create_track_segments(corners, braking_zones, accel_zones, track_length)

print(f"Created {len(segments)} track segments:")
for seg in segments:
    print(
        f"  [{seg.segment_type:12}] {seg.name:20} : {seg.start_dist:6.0f}m - {seg.end_dist:6.0f}m"
    )

Created 30 track segments:
  [braking     ] Turn 1 Braking       :    572m -    702m
  [corner      ] Turn 1               :    702m -    960m
  [acceleration] Turn 1 Exit          :    960m -   1192m
  [braking     ] Turn 2 Braking       :   1211m -   1236m
  [corner      ] Turn 2               :   1236m -   1398m
  [braking     ] Turn 3 Braking       :   1333m -   1433m
  [acceleration] Turn 2 Exit          :   1398m -   1564m
  [corner      ] Turn 3               :   1433m -   1862m
  [acceleration] Turn 3 Exit          :   1862m -   1881m
  [braking     ] Turn 4 Braking       :   1901m -   1926m
  [corner      ] Turn 4               :   1926m -   2136m
  [acceleration] Turn 4 Exit          :   2136m -   2664m
  [braking     ] Turn 5 Braking       :   2151m -   2251m
  [corner      ] Turn 5               :   2251m -   2358m
  [acceleration] Turn 5 Exit          :   2358m -   2664m
  [braking     ] Turn 6 Braking       :   2684m -   2780m
  [braking     ] Turn 7 Braking       :   268

In [10]:
# Visualize track segments on GPS map
# Convert lap_channels dict to DataFrame for plot_track_segments
import pandas as pd

lap_channels_df = pd.DataFrame(lap_channels)

fig = plot_track_segments(
    lap_channels_df,
    segments,
    title="Track Segments: Braking (Red), Corner (Orange), Acceleration (Green)",
)
show_fig(fig)